In [1]:
import dynamic_traffic_signal


In [2]:
a = dynamic_traffic_signal.DynamicTrafficSignal('Testing_class_ijipura')

In [3]:
a.optimize_traffic_lights()

15 25
Could not connect to TraCI server at localhost:59982 [Errno 111] Connection refused
 Retrying in 1 seconds
15 26
Could not connect to TraCI server at localhost:58863 [Errno 111] Connection refused
 Retrying in 1 seconds
15 27
Could not connect to TraCI server at localhost:54980 [Errno 111] Connection refused
 Retrying in 1 seconds
15 28
Could not connect to TraCI server at localhost:37196 [Errno 111] Connection refused
 Retrying in 1 seconds
15 29
Could not connect to TraCI server at localhost:43186 [Errno 111] Connection refused
 Retrying in 1 seconds
15 30
Could not connect to TraCI server at localhost:54246 [Errno 111] Connection refused
 Retrying in 1 seconds
15 31
Could not connect to TraCI server at localhost:51775 [Errno 111] Connection refused
 Retrying in 1 seconds
15 32
Could not connect to TraCI server at localhost:54943 [Errno 111] Connection refused
 Retrying in 1 seconds
15 33
Could not connect to TraCI server at localhost:43308 [Errno 111] Connection refused
 Retry

FatalTraCIError: connection closed by SUMO